# `scscrape` + `tweepy` approach
Using an alternative, indriect approach. Since Tweepy does not allow tweets older than 7 days, but can still return older results based on URLs. So we will use `snscrape` to get URLs of relevant tweent, and then feed these URLs to `tweepy`.


In [33]:
import tweepy
import pandas as pd

consumer_key = 'KpNlowpXetv4I8GwNxztcDdVa'
consumer_secret = '1B9RlLBtEgcRgWHphtpW0rCFsKb6meNPslM9V4SSR5VBHwEyMI'

auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Read the links to Twitter posts, obtained by snscrape 
links = pd.read_csv("data/snscrape-twitter-covid-Jan.txt", index_col= None, header = None, names = ["links"])
links.head()

,links
0,https://twitter.com/BastardTaitum/status/12125...
1,https://twitter.com/BastardTaitum/status/12125...
2,https://twitter.com/BastardTaitum/status/12124...
3,https://twitter.com/BastardTaitum/status/12124...
4,https://twitter.com/captmakret/status/12124955...


In [34]:
# Define a small one-liner function lamda which takes in x
# (which we pass onto the function) and splits all the entries
# under the heading "links" from the last, until the first '/'
trim = lambda x: x["links"].split("/")[-1]

# Apply the trimming horizontally (to every row of links)
links['id'] = links.apply(trim, axis=1)
print(links)

                                                  links                   id
0     https://twitter.com/BastardTaitum/status/12125...  1212510149170147328
1     https://twitter.com/BastardTaitum/status/12125...  1212509240830709761
2     https://twitter.com/BastardTaitum/status/12124...  1212499266968674304
3     https://twitter.com/BastardTaitum/status/12124...  1212499041625477122
4     https://twitter.com/captmakret/status/12124955...  1212495528791031810
...                                                 ...                  ...
4860  https://twitter.com/COVID_Careful/status/12226...  1222682396773904384
4861  https://twitter.com/COVID_Careful/status/12226...  1222682253852999680
4862  https://twitter.com/COVID_19Disease/status/122...  1222680220286799873
4863  https://twitter.com/COVID_JAX/status/122267653...  1222676535422877698
4864  https://twitter.com/___Covid_19___/status/1222...  1222674452875481091

[4865 rows x 2 columns]


In [35]:
# Divide into chunks of 100
chunk_size = 100
ids = links['id'].tolist()
total_count = len(ids)
chunks = (total_count // chunk_size) + 1
print(chunks, "chunks, each consisting 100 URLs")

49 chunks, each consisting 100 URLs


In [37]:
def get_info(ids):
    status_list = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in status_list:
            tweet_elem = {"tweet_id": status.id,
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at,
                     "likes":status.favorite_count,
                     "retweets":status.retweet_count}
            
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv("data/JanTweets.csv", mode="a")
    

for i in range(chunks):
    batch = ids[i*chunk_size : (i+1)*chunk_size]
    result = get_info(batch)
    print("Chunk ", i, " done")
         
print("Scrapping finished")


Chunk  0  done
Chunk  1  done
Chunk  2  done
Chunk  3  done
Chunk  4  done
Chunk  5  done
Chunk  6  done
Chunk  7  done
Chunk  8  done
Chunk  9  done
Chunk  10  done
Chunk  11  done
Chunk  12  done
Chunk  13  done
Chunk  14  done
Chunk  15  done
Chunk  16  done
Chunk  17  done
Chunk  18  done
Chunk  19  done
Chunk  20  done
Chunk  21  done
Chunk  22  done
Chunk  23  done
Chunk  24  done
Chunk  25  done
Chunk  26  done
Chunk  27  done
Chunk  28  done
Chunk  29  done
Chunk  30  done
Chunk  31  done
Chunk  32  done
Chunk  33  done
Chunk  34  done
Chunk  35  done
Chunk  36  done
Chunk  37  done
Chunk  38  done
Chunk  39  done
Chunk  40  done
Chunk  41  done
Chunk  42  done
Chunk  43  done
Chunk  44  done
Chunk  45  done
Chunk  46  done
Chunk  47  done
Chunk  48  done
Scrapping finished
